In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
#from keras.layers.recurrent import LSTM,GRU
#from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score
#from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

Using TensorFlow backend.


In [2]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0

--2020-03-06 04:28:19--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv [following]
--2020-03-06 04:28:20--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7d498a063b001bac8c5ef0d8a0.dl.dropboxusercontent.com/cd/0/inline/AzZ4Mnd8Varfrk8BhkHuqqaKBz-iTalZnd2VqHznE8YZt_eQu6G8lz1Ivz5LNg5mw42e9Lpj9HG-d5DzvUAWDOK-Jz6hYFgQ5FTCf8ZVlt3j3A/file# [following]
--2020-03-06 04:28:20--  https://uc7d498a063b001bac8c5ef0d8a0.dl.dropboxusercontent.com/cd/0/inline/AzZ4Mnd8Varfrk8BhkHuqqaKBz-iTalZnd2VqHznE8YZt_eQu6G8lz1Ivz

In [3]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2020-03-06 04:28:26--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-03-06 04:28:26--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-03-06 04:28:27--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [4]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [0]:
embedding_index ={}

f = open('/content/glove.42B.300d.txt', encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype ='float32')
  embedding_index[word] = coefs
f.close()

In [0]:
Data = pd.read_csv(r'/content/train.csv?dl=0')

In [7]:
print(Data.shape)
Data.head()

(1306122, 3)


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [8]:
X = Data['question_text']
Y = Data['target']
x_train, x_val, y_train, y_val = train_test_split(X,Y,test_size=0.2)
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((1044897,), (1044897,), (261225,), (261225,))

In [9]:
y_val.value_counts()

0    245029
1     16196
Name: target, dtype: int64

In [10]:
class_weight = class_weight.compute_class_weight('balanced', 
                                                 np.unique(y_train), 
                                                    y_train)
print(class_weight)
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

[0.53295681 8.08568576]


{0: 0.5329568094111599, 1: 8.085685764694958}

In [11]:
x_train

1151452                   Why are cardinal points important?
437557     Can you remove implicit bias from conversation...
91095      If your 18 in the army can you have your own p...
1289040          What happens if I hire a hitman to kill me?
316841                        Was the third reich an empire?
                                 ...                        
520203      How is double hashing applied in data structure?
1046189    How can I ask my doctor about prescribing noot...
583744     If I list my small business on Google My Busin...
368304     Is it common for men to feel the urge to rape ...
1267483    Corporate tax in KPMG, Mumbai or statutory aud...
Name: question_text, Length: 1044897, dtype: object

In [12]:
import nltk
nltk.download('punkt') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

361


In [14]:
print(np.percentile(sent_lens,99))
np.percentile(sent_lens,95)

45.0


30.0

In [15]:
max_len=45

tk=Tokenizer(char_level=False,split=' ')
tk.fit_on_texts(x_train)
print(x_train.shape)

(1044897,)


In [16]:
seq_train=tk.texts_to_sequences(x_train)
seq_val=tk.texts_to_sequences(x_val)
print(len(seq_val), len(seq_train))
seq_train[0]

261225 1044897


[16, 12, 15751, 1309, 215]

In [17]:
vocab_size=len(tk.word_index)
vocab_size

196227

In [18]:
seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_val_matrix=sequence.pad_sequences(seq_val,maxlen=max_len)
seq_train_matrix.shape, seq_val_matrix.shape

((1044897, 45), (261225, 45))

In [0]:
# creating our own embedding matrix to bring down the size to 300
# we'll use 300 D vector representation of the words from pretrained embedding index 
# that we downloaded 

embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embedding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector
        
# if there are specific words which are not present in pretrained embedding 
# their weights will remain 0. if there are too many such words 
# then you should probably not use pretrained embeddings 

In [20]:
# Now while creating the model for text data , we'll keep embedding layer weights 
# non-trainable as we are supplying those values from our end 

inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(layer1)
layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(layer2)
layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(layer3)
dense1 = Dense(25,activation='relu')(layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [21]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 45)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 45, 300)           58868400  
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3289

In [0]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [0]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [24]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [25]:
model.fit(seq_train_matrix,y_train,
              epochs=10,
              class_weight={0: 0.5328573351855138, 1: 8.108651115146436},
              batch_size=10000,
              validation_data=(seq_val_matrix,y_val),
              callbacks = [earlystop, checkpoint])



Train on 1044897 samples, validate on 261225 samples
Epoch 1/10
1044897/1044897 [==============================] - 782s 749us/step - loss: 0.3446 - acc: 0.8419 - val_loss: 0.2354 - val_acc: 0.9018

Epoch 00001: val_loss improved from inf to 0.23541, saving model to /content/weights-01-0.2354.h5
Epoch 2/10
1044897/1044897 [==============================] - 763s 730us/step - loss: 0.2844 - acc: 0.8797 - val_loss: 0.2950 - val_acc: 0.8782

Epoch 00002: val_loss did not improve from 0.23541
Epoch 3/10
1044897/1044897 [==============================] - 751s 719us/step - loss: 0.2641 - acc: 0.8888 - val_loss: 0.2462 - val_acc: 0.8943

Epoch 00003: val_loss did not improve from 0.23541
Epoch 4/10
1044897/1044897 [==============================] - 771s 738us/step - loss: 0.2493 - acc: 0.8944 - val_loss: 0.2878 - val_acc: 0.8797

Epoch 00004: val_loss did not improve from 0.23541
Epoch 00004: early stopping


In [0]:
best_model = load_model('/content/weights-01-0.2354.h5')

In [28]:
best_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 45)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 45, 300)           58868400  
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3289

In [30]:
p = best_model.predict(seq_val_matrix)
roc_auc_score(y_val,p)

0.9463256273894851

In [31]:
p

array([[0.0131745 ],
       [0.02104914],
       [0.051907  ],
       ...,
       [0.21765044],
       [0.06342798],
       [0.01940106]], dtype=float32)